##  import required libraries

In [1]:
# Importing Required Libraries
import numpy as np
import pandas as pd
import requests
from selenium import webdriver #importing web driver
from selenium.webdriver.chrome.service import Service # importting chrome service
from selenium.webdriver.chrome.options import Options # importing options to customize the behavior of the Chrome browser 
from selenium.webdriver.common.by import By # importing By to locate elements on a web page.
from bs4 import BeautifulSoup 
import time

## Summary

##### Here I am scrapping all billionaires data using this link "https://www.forbes.com/real-time-billionaires/#60825e283d78", but there is a problem here. 
1. I need a column industry but there is no specific column for industry. So I will have to manually scrape by each industry, so that I can create a new column called **Industry**.
2. In this billionaires data, female is also there but not specified in webpage. Here I will also manually scrape females name so that I can also create a column called **Sex** that will specify male and female billionaires.

##### Lets's Scrape by keeping this scenario.

In [3]:
s = Service('C:/Users/Vivek Kumar/Desktop/chromedriver.exe') # Importing chrome driver
driver = webdriver.Chrome(service=s)
driver.get('https://www.forbes.com/real-time-billionaires/')
driver.maximize_window()
time.sleep(5)

In [4]:
# Here, I have taken all the XPATH of industries and stored in a distionary for further use.
industry_path = {
    "Automotive":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[2]',
    "Construction & Engineering":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[3]',
    "Diversified":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[4]',
    "Energy":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[5]',
    "Fashion & Retail":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[6]',
    "Finance & Investments":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[7]',
    "Food & Beverage":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[8]',
    "Gambling & Casinos":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[9]',
    "Healthcare":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[10]',
    "Logistics":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[11]',
    "Manufacturing":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[12]',
    "Media & Entertainment":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[13]',
    "Metals & Mining":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[14]',
    "Real Estate":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[15]',
    "Service":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[16]',
    "Sports":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[17]',
    "Technology":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[18]',
    "Telecom":'/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/div/ul/li[19]'
}

In [5]:
# In our webpage, there are two scrollers, 'a scroler inside a scroller'. 
#So, Firstly I will locate outer scroller, then I will locate inner scroller, inside the outer scroller.
time.sleep(5)
outer_scroller = driver.find_element(by = By.XPATH, value = '//*[@id="ng-app"]')
driver.execute_script("arguments[0].scrollTo(0,arguments[0].scrollHeight);", outer_scroller)

In [6]:
for path in industry_path.items():
    # as i said above to scrape data by each industry, so here i will locate industry section from webpage.
    # Now, Lets collect all the industries's XPATH to fetch data.
    industry_dropdown = driver.find_element(by = By.XPATH, value = '/html/body/div[2]/div[2]/div/div[2]/div[1]/div/section[5]/div[2]/div/div[3]/div/div[1]/div[5]/span[2]')
    time.sleep(5)
    industry_dropdown.click()
    industry = driver.find_element(by = By.XPATH, value = path[1])
    industry.click() 
    
    # Searching for inner scroller in a outer scroller.
    inner_scroller = outer_scroller.find_element(by = By.XPATH, value = '//*[@id="row-4"]/div[2]/div/div[4]/div[1]')
    last_height = driver.execute_script("return arguments[0].scrollHeight;", inner_scroller)
    
    # Running a Infinite Loop
    for i in iter(int, 1):
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", inner_scroller)
        time.sleep(10)
        new_height = driver.execute_script("return arguments[0].scrollHeight;", inner_scroller)
        
        
        # If last height and new height will be same, Break the infinite loop
        if new_height == last_height:
            break
        last_height = new_height
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    
    
    image = []
    rank = []
    name = []
    Net_Worth = []
    age = []
    source = []
    country_territory = []
    Industry = []
    link = []

    for i in soup.find_all('tr', 'ng-scope'):
        try:
            image.append(i.find('img')['src']) # image
        except:
            image.append('NaN')
        try:    
            rank.append(i.find('td','rank').find('span','ng-binding').text) #rank
        except:
            rank.append("NaN")
        try:    
            name.append(i.find('td','name').h3.text) #name
        except:
            name.append("NaN")
        try:    
            Net_Worth.append(i.find('td','Net').text) #net_worth   
        except:
            Net_Worth.append("NaN")    
        try:    
            age.append(i.find('td','age').text) #age 
        except:
            age.append("NaN")

        try:    
            source.append(i.find('td','source').text) # source    
        except:
            source.append("NaN")     

        try:    
            country_territory.append(i.find('td','Country/Territory').text) # Country
        except:
            country_territory.append("NaN")  
            
        try:
            link.append(i.find('h3').a.get('href')) # Name's internal link to scrape more information again.
        except:
            link.append(np.nan)
    
    # If path's name is "Automotive", Create a DataFrame
    if path[0] == 'Automotive':
        x = pd.DataFrame({
            "Image":image,
            "Rank":rank,
            "Name":name,
            "Net_Worth":Net_Worth,
            "Age":age,
            "Source":source,
            "Country":country_territory,
            "internal_link":link,
            "Industry":path[0]
       })
        
    # If path is other than automotive, concat the below dataframe to variable x    
    else:
        x = pd.concat([x,
                  pd.DataFrame({
                    "Image":image,
                    "Rank":rank,
                    "Name":name,
                    "Net_Worth":Net_Worth,
                    "Age":age,
                    "Source":source,
                    "Country":country_territory,
                    "internal_link":link,
                    "Industry":path[0]
                 }) ], ignore_index=True)
        
print("Done")        

Done


In [7]:
x.to_csv("billionare_file.csv", encoding = 'utf-8')

## Now lets extract female's billionares list

In [8]:
s = Service('C:/Users/Vivek Kumar/Desktop/chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get('https://www.forbes.com/real-time-billionaires/#42e2c8563d78')
driver.maximize_window()

In [9]:
# Locating Female XPATH.
female_page = driver.find_element(by = By.XPATH, value= '//*[@id="row-4"]/div[2]/div/div[3]/div/div[1]/div[4]')
female_page.click()

In [10]:
time.sleep(5)
# Locate for outer scroller first, if you want to locate inner scroller
outer_scroller = driver.find_element(by = By.XPATH, value = '//*[@id="ng-app"]')
# scroll to end
driver.execute_script("arguments[0].scrollTo(0,arguments[0].scrollHeight);", outer_scroller)

In [11]:
time.sleep(5)
# Locating inner scroller inside outer scroler
inner_scroller = outer_scroller.find_element(by = By.XPATH, value = '//*[@id="row-4"]/div[2]/div/div[4]/div[1]')
# calculate webpage height before scrolling
last_height = driver.execute_script("return arguments[0].scrollHeight;", inner_scroller)
# running a infinite loop
for i in iter(int, 1):
    #scrolling inner scroller to infinite to fetch html page.
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", inner_scroller)
    time.sleep(10)
    new_height = driver.execute_script("return arguments[0].scrollHeight;", inner_scroller)

    if new_height == last_height:
        break
    last_height = new_height

In [12]:
# Extracting all html file
html = driver.page_source
soup = BeautifulSoup(html)

In [13]:
soup.find('h3').a.get('href')

'https://www.forbes.com/profile/francoise-bettencourt-meyers/?list=rtb/'

In [14]:
female_name = []
for i in soup.find_all('tr', 'ng-scope'):
    try:    
        female_name.append(i.find('td','name').h3.text)    
    except:
        female_name.append(np.nan)

In [15]:
x = pd.DataFrame({
    'female_name':female_name
})

In [19]:
x.to_csv('female_billionares.csv', encoding = 'utf-8')

##### So I scraped all the data from this link "https://www.forbes.com/real-time-billionaires/#3b1023cd3d78", which include males data and females data both. Further I will merge both the datasets and will clean it.

# Thank You